## Hausarbeit 7
• Modifizieren Sie das Modell so, dass es einen 5x5 Kernel verwendet <br>
    1. Wie verändert sich dann die Anzahl der freien Parameter im Modell? <br>
    2. Verbessert oder verschlechtert sich Overfitting? <br>
    
• Können Sie ein Bild finden, welches weder Vogel noch Flugzeug <br>
enthält, von dem Netz aber mit mindestens 95% wahrscheinlichkeit 
als eines der beiden Objekte erkannt wird?


<span style="color:red">**Anmerkung: Die Beantwortung der Fragen finden Sie im Verlauf des Notebooks**</span>

In [138]:
# Laden der notwendigen Bibliotheken
from matplotlib import pyplot as plt
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import os
import collections
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import datetime 

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [139]:
# Definition der einzelnen Klassen
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [140]:
# Laden des Datensatzes: Umwandlung in einen Tensor und Normalisierung der Daten
data_path = '../data-unversioned/p1ch6/'

# Trainingsdatensatz
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

# Validierungsdatensatz
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified
Files already downloaded and verified


In [141]:
# Bildung eines neuen Datensatzes, nur mit Bird und Flugzeug Bildern
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']

# Trainingsdatensatz
cifar2 = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 2]]

# Validierungsdatensatz
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

#### 1. Aufgabe: Vergleich 5x5 und 3x3 Kernel

### a.) 3x3 Kernel:

In [142]:
# Definition des Baseline-Modells:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Definition eines Convolutional-Layers mit einem 3x3 Kernel
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [143]:
# Definition des Trainingloops
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):  
        loss_train = 0.0
        for imgs, labels in train_loader:  
            
            outputs = model(imgs)  
            
            loss = loss_fn(outputs, labels)  

            optimizer.zero_grad()  
            
            loss.backward()  
            
            optimizer.step()  

            loss_train += loss.item()  

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader))) 

In [144]:
# Trainieren des Modells
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=True) 

model = Net()  
optimizer = optim.SGD(model.parameters(), lr=1e-2)  
loss_fn = nn.CrossEntropyLoss()  

training_loop(  
    n_epochs = 30,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

print('Die Anzahl der freien Parameter beträgt:', sum(p.numel() for p in model.parameters()))

2023-05-31 14:03:20.466169 Epoch 1, Training loss 0.5803694548500571
2023-05-31 14:03:40.432775 Epoch 10, Training loss 0.33222396576860147
2023-05-31 14:04:02.247335 Epoch 20, Training loss 0.2911597882296629
2023-05-31 14:04:24.764012 Epoch 30, Training loss 0.2659074995358279


18090

In [145]:
# Berechnung der Genauigkeit des Modells für die Trainings und Validierungsdaten:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64,
                                         shuffle=False)
all_acc_dict = collections.OrderedDict()

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():  
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <2>
                total += labels.shape[0]  # <3>
                correct += int((predicted == labels).sum())  # <4>

        print("Accuracy {}: {:.2f}".format(name , correct / total))


all_acc_dict["baseline"] = validate(model, train_loader, val_loader)


Accuracy train: 0.88
Accuracy val: 0.87


### b.) 5x5 Kernel

In [150]:
# Definition des Baseline-Modells:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Definition eines Convolutional-Layers mit einem 3x3 Kernel
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=5, padding=2)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

# Trainieren des Modells
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=True) 

model = Net()  
optimizer = optim.SGD(model.parameters(), lr=1e-2)  
loss_fn = nn.CrossEntropyLoss()  

training_loop(  
    n_epochs = 30,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

print('Die Anzahl der freien Parameter mit dem 5x5 Kernel beträgt:', sum(p.numel() for p in model.parameters()))

2023-05-31 14:16:41.204586 Epoch 1, Training loss 0.5651234084633505
2023-05-31 14:17:03.532265 Epoch 10, Training loss 0.3220862958841263
2023-05-31 14:17:53.036285 Epoch 20, Training loss 0.2858740151118321
2023-05-31 14:18:57.599060 Epoch 30, Training loss 0.25616376989396517
Die Anzahl der freien Parameter mit dem 5x5 Kernel beträgt: 20906
